In [8]:
import os
from pathlib import Path
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB7
from tensorflow.keras import layers

path = '../mergedData/'

image_gen = ImageDataGenerator(validation_split=0.2)
train_generator = image_gen.flow_from_directory(
    path + "Training", target_size=(224, 224),batch_size=64, class_mode='categorical', subset='training')
val_generator = image_gen.flow_from_directory(
    path + "Training", target_size=(224, 224),batch_size=64, class_mode='categorical', subset='validation')

test_generator = image_gen.flow_from_directory(
    path + "Test", target_size=(224, 224),batch_size=64, class_mode='categorical')


Found 31628 images belonging to 67 classes.
Found 7885 images belonging to 67 classes.
Found 16999 images belonging to 67 classes.


In [9]:
print(train_generator.class_indices)
print(len(train_generator.class_indices))

{'Apple': 0, 'Apricot': 1, 'Avocado': 2, 'Banana': 3, 'Beetroot': 4, 'Blueberry': 5, 'Cactus': 6, 'Cantaloupe': 7, 'Carambula': 8, 'Cauliflower': 9, 'Cherry': 10, 'Chestnut': 11, 'Clementine': 12, 'Cocos': 13, 'Corn': 14, 'Cucumber': 15, 'Dates': 16, 'Eggplant': 17, 'Fig': 18, 'Ginger': 19, 'Granadilla': 20, 'Grape': 21, 'Grapefruit': 22, 'Guava': 23, 'Hazelnut': 24, 'Huckleberry': 25, 'Kaki': 26, 'Kiwi': 27, 'Kohlrabi': 28, 'Kumquats': 29, 'Lemon': 30, 'Limes': 31, 'Lychee': 32, 'Mandarine': 33, 'Mango': 34, 'Mangostan': 35, 'Maracuja': 36, 'Melon': 37, 'Mulberry': 38, 'Nectarine': 39, 'Nut': 40, 'Onion': 41, 'Orange': 42, 'Papaya': 43, 'Passion': 44, 'Peach': 45, 'Pear': 46, 'Pepino': 47, 'Pepper': 48, 'Physalis': 49, 'Pineapple': 50, 'Pitahaya': 51, 'Plum': 52, 'Pomegranate': 53, 'Pomelo': 54, 'Potato': 55, 'Quince': 56, 'Rambutan': 57, 'Raspberry': 58, 'Redcurrant': 59, 'Salak': 60, 'Strawberry': 61, 'Tamarillo': 62, 'Tangelo': 63, 'Tomato': 64, 'Walnut': 65, 'Watermelon': 66}
67


In [10]:
#    Convolution
#    Non Linearity (ReLU)
#    Pooling or Sub Sampling
#    Classification (Fully Connected Layer)
#    Peamised põhilayerid

In [11]:
inputs = layers.Input(shape=(224, 224, 3))
#x = img_augmentation(inputs)
model = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=inputs, classes=len(train_generator.class_indices))

model.trainable = False
x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(len(train_generator.class_indices), activation="softmax", name="pred")(x)

model = tf.keras.Model(inputs, outputs)

for layer in model.layers[-20:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

In [12]:
seed = 1111

model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_2 (Rescaling)         (None, 224, 224, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
normalization_2 (Normalization) (None, 224, 224, 3)  7           rescaling_2[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_2[0][0]            
_______________________________________________________________________________________

In [14]:
#optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(train_generator, epochs=10, validation_data=val_generator, verbose=1)
results = model.evaluate(test_generator)
print(results)

Epoch 1/10
495/495 [==============================] - 1278s 3s/step - loss: 0.1022 - accuracy: 0.9753 - val_loss: 0.0383 - val_accuracy: 0.9915
Epoch 2/10
495/495 [==============================] - 1296s 3s/step - loss: 0.0128 - accuracy: 0.9969 - val_loss: 0.0567 - val_accuracy: 0.9835
Epoch 3/10
495/495 [==============================] - 1286s 3s/step - loss: 0.0088 - accuracy: 0.9978 - val_loss: 0.1254 - val_accuracy: 0.9727
Epoch 4/10
495/495 [==============================] - 1277s 3s/step - loss: 0.0155 - accuracy: 0.9962 - val_loss: 0.0337 - val_accuracy: 0.9918
Epoch 5/10
495/495 [==============================] - 1276s 3s/step - loss: 0.0122 - accuracy: 0.9967 - val_loss: 0.1683 - val_accuracy: 0.9715
Epoch 6/10
495/495 [==============================] - 1277s 3s/step - loss: 0.0110 - accuracy: 0.9968 - val_loss: 0.0602 - val_accuracy: 0.9848
Epoch 7/10
495/495 [==============================] - 1286s 3s/step - loss: 0.0092 - accuracy: 0.9973 - val_loss: 0.0791 - val_accuracy:

In [15]:
!mkdir -p saved_model
model.save('saved_model/effcient_net') 
new_model = tf.keras.models.load_model('saved_model/effcient_net')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/effcient_net/assets


In [69]:
phone_img_gen = image_gen.flow_from_directory(
    path + "single_test/Apelsin", target_size=(224, 224),batch_size=1, class_mode='categorical')
results = new_model.predict(phone_img_gen)


Found 1 images belonging to 1 classes.


In [71]:
#print(results[0])
print(results[0])
print(results[0][41])
print(np.sum(results[0]))
print("---------")
#print(results[1])
#print(results[1][41])
#print(np.sum(results[1]))
#print("---------")
#print(results[2])
#print(results[2][3])
#print(np.sum(results[2]))
#print("---------")
results.argmax(axis=-1)

[2.66293959e-10 1.25643228e-11 4.76898480e-04 2.51912559e-13
 3.17667490e-11 1.45356213e-10 1.40663543e-15 1.16138972e-05
 2.85414886e-11 4.75402004e-07 1.53757530e-12 1.61124217e-12
 1.51913898e-10 1.71752196e-12 6.15250201e-06 5.66523336e-13
 3.70430409e-10 2.49647608e-10 5.71098963e-15 4.06797236e-13
 1.52703151e-07 1.36397327e-09 1.35442484e-02 1.75758700e-13
 1.06989111e-08 4.49648185e-14 1.14209667e-13 2.88677682e-09
 5.89748583e-10 1.38971714e-08 1.45766077e-09 6.52044490e-02
 1.51944041e-10 1.41361286e-14 1.47709319e-12 4.71300042e-14
 1.35845016e-10 2.61920814e-08 9.06171351e-08 4.13260937e-09
 1.31835942e-09 1.01138431e-13 9.07599866e-01 2.93141900e-10
 7.49336926e-10 2.42749258e-11 1.97158487e-11 2.34298136e-11
 3.52692209e-09 9.89386863e-14 1.05598585e-11 3.63810788e-13
 1.77283146e-10 1.66460135e-14 1.31246569e-02 2.76121503e-09
 3.51205257e-13 1.84875240e-10 4.16304803e-11 1.71489667e-12
 5.71246141e-11 3.01718828e-05 3.13200403e-11 4.46095357e-14
 3.85296905e-10 1.033330

array([42])

In [64]:
newresults = model.evaluate(test_generator)
print(results)

266/266 [==============================] - 593s 2s/step - loss: 0.3475 - accuracy: 0.9498
[[1.88449165e-04 1.18725429e-05 5.91067590e-07 1.70532672e-08
  4.95141030e-05 9.17277987e-08 9.31506174e-12 1.59771957e-07
  6.23948324e-07 8.63845289e-01 4.26232930e-07 1.93095616e-07
  1.39833367e-09 4.52617940e-07 4.27707797e-03 2.19787341e-10
  7.47605282e-06 4.86568133e-05 8.80747628e-11 1.11467096e-04
  5.37376865e-10 1.71523045e-08 2.86990660e-03 1.61471959e-07
  4.78992579e-09 1.88330818e-09 7.82035325e-10 6.36480672e-06
  7.92937651e-02 2.21266589e-07 1.62547522e-08 1.22585570e-05
  2.46781383e-07 1.03239346e-08 1.94637551e-07 3.46365846e-06
  6.46321574e-09 3.42942403e-05 8.42190548e-06 1.80292147e-06
  6.07740822e-06 9.71949630e-05 4.10079374e-05 3.72073742e-07
  1.57039995e-05 7.00282408e-06 3.18667475e-08 3.48474856e-07
  4.47373837e-03 5.65296165e-09 1.37229566e-04 4.07888547e-05
  4.25640121e-02 3.31341857e-06 5.27087877e-07 3.02061162e-05
  1.14715315e-09 5.52620797e-04 2.09131365